In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

In [2]:
# all asume that values start from row 2 (i.e., 1st row is header)
def filter_deltaAnnot(data, tol):
    return np.where(np.abs(data.iloc[2:,2])>tol)[0]+2

# inplace transform
def fill_gap_nan_(data, value):
    data.iloc[2:,88:].fillna(value, inplace=True)

# inplace transform
def log2_transform_(data):
    for i in range(2,data.shape[0]):
        for j in range(8,48):
            data.iloc[i,j]=np.log2(data.iloc[i,j])
    
def get_VAL_duplicates(data, tol):
    indices = []
    
    for i in range(2,data.shape[0]-1):
        a = np.asarray(data.iloc[i,8:48], dtype=float)
        for j in range(i+1,data.shape[0]):
            b = np.asarray(data.iloc[j,8:48], dtype=float)
            if(np.sum(a==b)/len(a) >= tol):
                indices.append((i,j))
    result=np.zeros((len(indices),2))
    for i in range(result.shape[0]):
        result[i,0]=indices[i][0]
        result[i,1]=indices[i][1]
    return result

@jit(nopython=True)
def duplicates(data, tol):
    n = data.shape[0]
    result = np.zeros((n+2,))
    
    for i in range(n-1):
        a = data[i,:]
        for j in range(i+1,n):
            b = data[j,:]
            if(np.sum(a==b)/len(a) >= tol):
                result[i+2]=1
                result[j+2]=1
                #continue
                
    return result, np.array([i for i in np.arange(n) if result[i]])

def get_VAL_duplicates_fast(data, tol):
    indices = []
    
    for i in range(2,data.shape[0]-1):
        a = np.asarray(data.iloc[i,8:48], dtype=float)
        for j in range(i+1,data.shape[0]):
            b = np.asarray(data.iloc[j,8:48], dtype=float)
            if(np.sum(a==b)/len(a) >= tol):
                indices.append((i,j))
                continue
    result=np.zeros((len(indices),2))
    for i in range(result.shape[0]):
        result[i,0]=indices[i][0]
        result[i,1]=indices[i][1]
    return np.asarray(result,dtype=int).flatten()

def ind2bool(inds,n):
    lam = lambda i: i in inds
    result = np.array([lam(i) for i in range(n)])
    return result
    

def check_reliability(data, tol=0.8, columns= np.array(range(88,128)),acceptable=np.array([0,64,128])):
    tol=0.8
    n=columns.shape[0]
    accept= np.array([0,64,128])
    keep=[]
    for i in range(2,data.shape[0]):
        a=np.asarray(data.iloc[i,columns],dtype=int)
        s=0
        for elem in a:
            s+=int(elem in accept)
        if(s>=tol*n):
            keep.append(i)
    return np.array(keep)
    

### Manually first prune QCs out of every sample

This allows you to slice 8:48 sample ID's, and then use those to get matching gp_status and gp_method columns. Also check that the order is correct.

### List of all files

In [3]:
path = '../PDproj/celldata/datawithGFstatus/original/'
path_out = '../PDproj/celldata/datawithGFstatus/clean/v2/'
fname = 'cellshilicnegGFstatus.xlsx'
filenames = [line.rstrip() for line in open(path +'filenames.txt')]
filenames

['cellshilicnegGFstatus.xlsx',
 'cellshilicposGFstatus.xlsx',
 'cellslipidnegGFstatus.xlsx',
 'cellslipidposGFstatus.xlsx',
 'cellsRPnegGFstatus.xlsx',
 'cellsRPposGFstatus.xlsx',
 'mediumhilicnegGFstatus.xlsx',
 'mediumhilicposGFstatus.xlsx',
 'mediumRPnegGFstatus.xlsx',
 'mediumRPposGFstatus.xlsx']

### all files at once

In [77]:
for fnum in range(10):
    print("Now processing {}".format(filenames[fnum]))
    data = pd.read_excel(path + filenames[fnum])
    colnames = data.columns
    
    th=1000
    #drop duplicates with high deltamass
    #dup=get_VAL_duplicates_fast(data,tol=1)
    #to_drop=np.array(list(set( np.unique(dup.flatten())) & set(filter_deltaAnnot(data,th))), dtype=int)
    
    # drop all with high deltamass
    to_drop=filter_deltaAnnot(data,th)
    data=data.drop(filter_deltaAnnot(data,th),axis=0)
    
    IDs=[]
    for s in data.iloc[1,8:]:
        #print(s)
        IDs.append(int(s.split('_')[-1].split('.')[0]))

    IDs=np.array(IDs)
    area_id = IDs[:40]
    GAP_id = IDs[40:]
    bools = np.asarray([GAP_id[i] in area_id for i in range(len(GAP_id))], dtype=int)
    bools=np.hstack([np.ones((48,)),bools])
    keep_ind = np.array([i for i in range(len(bools)) if bools[i]])
    print("Keep 128 columns: {}".format(keep_ind.shape[0]==128))
    
    data = data.iloc[:,keep_ind]
    print('Number of groups: {} (40 in each area, gp method, gp status)'.format((data.shape[1]-8)/40))

    IDs=[]
    for s in data.iloc[1,8:]:
        #print(s)
        IDs.append(int(s.split('_')[-1].split('.')[0]))

    IDs=np.array(IDs)
    area_id = IDs[:40]
    GAP_id = IDs[40:]

    print("Order matches: {}".format(np.alltrue(area_id==GAP_id[:40]) and np.alltrue(area_id==GAP_id[40:])))
    print("Number of unique ID's: {}".format(np.unique(IDs).shape[0]))
    
    
    log2_transform_(data)
    fill_gap_nan_(data,0)
    
    new_colnames=['aSYN' for i in range(10)] + ['comb.' for i in range(10)] + ['INFg' for i in range(10)] + ['UT' for i in range(10)]
    new_colnames=['' for i in range(8)] + new_colnames + ['' for i in range(80)]
    data.columns=np.array(new_colnames)
    
    fname_out = path_out + filenames[fnum].split('.')[0] + '.csv'
    data.to_csv(fname_out, header=True, index=False, sep=';')
    

Now processing cellshilicnegGFstatus.xlsx
Keep 128 columns: True
Number of groups: 3.0 (40 in each area, gp method, gp status)
Order matches: True
Number of unique ID's: 40
Now processing cellshilicposGFstatus.xlsx
Keep 128 columns: True
Number of groups: 3.0 (40 in each area, gp method, gp status)
Order matches: True
Number of unique ID's: 40
Now processing cellslipidnegGFstatus.xlsx
Keep 128 columns: True
Number of groups: 3.0 (40 in each area, gp method, gp status)
Order matches: True
Number of unique ID's: 40
Now processing cellslipidposGFstatus.xlsx
Keep 128 columns: True
Number of groups: 3.0 (40 in each area, gp method, gp status)
Order matches: True
Number of unique ID's: 40
Now processing cellsRPnegGFstatus.xlsx
Keep 128 columns: True
Number of groups: 3.0 (40 in each area, gp method, gp status)
Order matches: True
Number of unique ID's: 40
Now processing cellsRPposGFstatus.xlsx
Keep 128 columns: True
Number of groups: 3.0 (40 in each area, gp method, gp status)
Order matches:

### Select a file

In [4]:
fnum=0
data = pd.read_excel(path + filenames[fnum])
colnames = data.columns
filenames[fnum]

'cellshilicnegGFstatus.xlsx'

### Drop rows with delta Annot over a threshold

In [167]:
b,i=duplicates(data.iloc[2:,8:48].to_numpy(dtype=float),1)

In [5]:
dup=get_VAL_duplicates(data,tol=1)
to_drop=np.array(list(set( np.unique(dup.flatten())) & set(filter_deltaAnnot(data,1000))), dtype=int)
to_drop

array([], dtype=int64)

In [7]:
to_drop=filter_deltaAnnot(data,1000)
to_drop

array([ 81, 117])

In [49]:
th=1000
data=data.drop(to_drop,axis=0)

### Align the id's and get indices to prune out excess columns

In [50]:
IDs=[]
for s in data.iloc[1,8:]:
    #print(s)
    IDs.append(int(s.split('_')[-1].split('.')[0]))

IDs=np.array(IDs)
area_id = IDs[:40]
GAP_id = IDs[40:]
bools = np.asarray([GAP_id[i] in area_id for i in range(len(GAP_id))], dtype=int)
bools=np.hstack([np.ones((48,)),bools])
keep_ind = np.array([i for i in range(len(bools)) if bools[i]])
print("Keep 128 columns: {}".format(keep_ind.shape[0]==128))

Keep 128 columns: True


### Remove the excess columns (id's) and check that the result is correct

In [51]:
data = data.iloc[:,keep_ind]
print('Number of groups: {} (40 in each area, gp method, gp status)'.format((data.shape[1]-8)/40))

IDs=[]
for s in data.iloc[1,8:]:
    #print(s)
    IDs.append(int(s.split('_')[-1].split('.')[0]))

IDs=np.array(IDs)
area_id = IDs[:40]
GAP_id = IDs[40:]

print("Order matches: {}".format(np.alltrue(area_id==GAP_id[:40]) and np.alltrue(area_id==GAP_id[40:])))
print("Number of unique ID's: {}".format(np.unique(IDs).shape[0]))

Number of groups: 3.0 (40 in each area, gp method, gp status)
Order matches: True
Number of unique ID's: 40


### Log transform (inplace) and fill nan's (inplace)

In [52]:
log2_transform_(data)
fill_gap_nan_(data,0)

### Reliability:

In [228]:
#keep_rows=check_reliability(data,columns=np.array(range(88,98)))

### Find value duplicates

In [229]:
#duplicate_ind=get_VAL_duplicates(data,1)
#duplicate_ind

In [79]:
#data.iloc[[13,49]]

**Correct the columnnames:**

In [53]:
new_colnames=['aSYN' for i in range(10)] + ['comb.' for i in range(10)] + ['INFg' for i in range(10)] + ['UT' for i in range(10)]
new_colnames=['' for i in range(8)] + new_colnames + ['' for i in range(80)]
data.columns=np.array(new_colnames)

**And save to file named:**

In [54]:
fname_out = path_out + filenames[fnum].split('.')[0] + '.csv'
fname_out

'../PDproj/celldata/datawithGFstatus/clean/v2/cellshilicnegGFstatus.csv'

### Save

In [55]:
data.to_csv(fname_out, header=True, index=False, sep=';')

In [3]:
path = '../PDproj/celldata/datawithGFstatus/clean/'
filenames = [line.rstrip() for line in open(path +'filenames.txt')]
filenames

['cellshilicnegGFstatus.csv',
 'cellshilicposGFstatus.csv',
 'cellslipidnegGFstatus.csv',
 'cellslipidposGFstatus.csv',
 'cellsRPnegGFstatus.csv',
 'cellsRPposGFstatus.csv',
 'mediumhilicnegGFstatus.csv',
 'mediumhilicposGFstatus.csv',
 'mediumRPnegGFstatus.csv',
 'mediumRPposGFstatus.csv']